In [1]:
import pandas as pd
# import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import gc

In [2]:
train = pd.read_csv('training_dataset/hourly_dataset.csv')
train['time'] = pd.to_datetime(train['time'])

In [3]:
train1 = train[(train['time']>='2022-01-01 01:00:00')&(train['time']<'2022-05-01 01:00:00')].reset_index(drop=True)
# test1 = train[train['train or test']=='test1'].reset_index(drop=True)

train2 = train[(train['time']>='2022-05-08 01:00:00')&(train['time']<'2022-06-01 01:00:00')].reset_index(drop=True)
# test2 = train[train['train or test']=='test2'].reset_index(drop=True)

train3 = train[(train['time']>='2022-06-08 01:00:00')&(train['time']<'2022-07-21 01:00:00')].reset_index(drop=True)
# test3 = train[train['train or test']=='test3'].reset_index(drop=True)

train4 = train[(train['time']>='2022-07-28 01:00:00')&(train['time']<'2022-08-21 01:00:00')].reset_index(drop=True)
# test4 = train[train['train or test']=='test4'].reset_index(drop=True)

In [4]:
## 将小于零的值替换为空值
for df in tqdm([train1,train2,train3,train4]):
    for i in range(1,21):
        df.loc[df[f'flow_{i}']<0,f'flow_{i}'] = np.nan
        
        
## 将异常值替换为空值
for df in tqdm([train1,train2,train3,train4]):
    for i in range(1,21):
        df[f'flow_{i}_shift'] = df[f'flow_{i}'].shift(-1)
        for index,row in df.iterrows():
            if index != df.shape[0]-1:
                if pd.isna(row[f'flow_{i}_shift']) and pd.notna(row[f'flow_{i}']):
                    df.loc[index,f'flow_{i}'] = np.nan
        del df[f'flow_{i}_shift'];gc.collect

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
for i in range(1,5):
    locals()[f'train{i}'] = locals()[f'train{i}'].set_index('time')
    
for df in [train1,train2,train3,train4]:
    for date in tqdm(df.index):
        for col in range(20):
            period1 = pd.Timedelta(hours = 24)
            if pd.isna(df.loc[date][col]): #判断是否为空值
                if date - period1 in df.index: #如果该索引存在 （考虑到一下flow一开始就存在空值）
                    period2 = date - period1
                    while pd.isna(df.loc[period2][col]):
                        period2 = period2 - period1
                    df.loc[date, f'flow_{col+1}'] = df.loc[period2, f'flow_{col+1}']

                else: #对于flow一开始就存在缺失值的，用后一天的值进行填充
                    period2 = date + pd.Timedelta(hours = 24)
                    while pd.isna(df.loc[period2][col]):
                        period2 = period2 + period1
                    df.loc[date, f'flow_{col+1}'] = df.loc[period2, f'flow_{col+1}']

                
for i in range(1,5):
    locals()[f'train{i}'] = locals()[f'train{i}'].reset_index()

  0%|          | 0/2880 [00:00<?, ?it/s]

  0%|          | 0/576 [00:00<?, ?it/s]

  0%|          | 0/1032 [00:00<?, ?it/s]

  0%|          | 0/576 [00:00<?, ?it/s]

In [6]:
sub = pd.read_csv('sample_submission.csv')

# sub1 = (train1.iloc[-24*7*1:,1:-1].values + train1.iloc[-24*7*2:-24*7*1,1:-1].values + \
#         train1.iloc[-24*7*3:-24*7*2,1:-1].values + train1.iloc[-24*7*4:-24*7*3,1:-1].values)/4
# sub.loc[(sub['time']>='2022-05-01 01:00:00')&(sub['time']<'2022-05-08 01:00:00'),1:] = sub1


# sub1 = (train2.iloc[-24*7*1:,1:-1].values + train2.iloc[-24*7*2:-24*7*1,1:-1].values + \
#         train2.iloc[-24*7*3:-24*7*2,1:-1].values)/3
# sub.loc[(sub['time']>='2022-06-01 01:00:00')&(sub['time']<'2022-06-08 01:00:00'),1:] = sub1


# sub1 = (train3.iloc[-24*7*1:,1:-1].values + train3.iloc[-24*7*2:-24*7*1,1:-1].values + \
#         train3.iloc[-24*7*3:-24*7*2,1:-1].values + train3.iloc[-24*7*4:-24*7*3,1:-1].values)/4
# sub.loc[(sub['time']>='2022-07-21 01:00:00')&(sub['time']<'2022-07-28 01:00:00'),1:] = sub1


# sub1 = (train4.iloc[-24*7*1:,1:-1].values + train4.iloc[-24*7*2:-24*7*1,1:-1].values + \
#         train4.iloc[-24*7*3:-24*7*2,1:-1].values)/3
# sub.loc[(sub['time']>='2022-08-21 01:00:00')&(sub['time']<'2022-08-28 01:00:00'),1:] = sub1
sub1 = ((train1.iloc[-24*7*1:-24*6,1:-1].values + train1.iloc[-24*7*2:-24*7*1-24*6,1:-1].values + \
         train1.iloc[-24*7*3:-24*7*2-24*6,1:-1].values + train1.iloc[-24*7*4:-24*7*3-24*6,1:-1].values)/4)\
             * 0.2 + train1.iloc[-24*1:,1:-1].values * 0.8
sub.loc[(sub['time']>='2022-05-01 01:00:00')&(sub['time']<'2022-05-02 01:00:00'),1:] = sub1
for i in range(2, 8):
    if i == 7 :
        sub.iloc[24*(i-1)*1:24*(i),1:] = ((train1.iloc[-24*1:,1:-1].values + train1.iloc[-24*7*1-24*1:-24*7*1,1:-1].values +\
            + train1.iloc[-24*7*2-24*1:-24*7*2,1:-1].values + train1.iloc[-24*7*3-24*1:-24*7*3,1:-1].values)/4)\
                 * 0.4 + sub.iloc[24*(i-2)*1:24*(i-1),1:].values * 0.6
    else:
        sub1 = ((train1.iloc[-24*(8-i)*1:-24*(7-i),1:-1].values + train1.iloc[-24*7*1-24*(8-i)*1:-24*7*1-24*(7-i),1:-1].values +\
            train1.iloc[-24*7*2-24*(8-i)*1:-24*7*2-24*(7-i),1:-1].values + train1.iloc[-24*7*3-24*(8-i)*1:-24*7*3-24*(7-i),1:-1].values)/4)\
                 * 0.4 + sub.iloc[24*(i-2)*1:24*(i-1),1:].values * 0.6
        sub.iloc[24*(i-1)*1:24*(i),1:] = sub1

sub2 = ((train2.iloc[-24*7*1:-24*6,1:-1].values + train2.iloc[-24*7*2:-24*7*1-24*6,1:-1].values + \
         train2.iloc[-24*7*3:-24*7*2-24*6,1:-1].values)/3)\
             * 0.2 + train2.iloc[-24*1:,1:-1].values * 0.8
sub.loc[(sub['time']>='2022-06-01 01:00:00')&(sub['time']<'2022-06-02 01:00:00'),1:] = sub2
for i in range(2, 8):
    if i == 7 :
        sub.iloc[24*7*1 + 24*(i-1)*1:24*7 + 24*(i),1:] = ((train2.iloc[-24*1:,1:-1].values + train2.iloc[-24*7*1-24*1:-24*7*1,1:-1].values +\
            + train2.iloc[-24*7*2-24*1:-24*7*2,1:-1].values)/3)\
                 * 0.4 + sub.iloc[24*7*1 + 24*(i-2)*1:24*7*1 + 24*(i-1),1:].values * 0.6
    else:
        sub.iloc[24*7*1 + 24*(i-1)*1:24*7*1 + 24*(i),1:] = ((train2.iloc[-24*(8-i)*1:-24*(7-i),1:-1].values + train2.iloc[-24*7*1-24*(8-i)*1:-24*7*1-24*(7-i),1:-1].values +\
            train2.iloc[-24*7*2-24*(8-i)*1:-24*7*2-24*(7-i),1:-1].values)/3)\
                 *0.4 + sub.iloc[24*7*1 + 24*(i-2)*1:24*7*1 + 24*(i-1),1:].values * 0.6
        
sub3 = ((train3.iloc[-24*7*1:-24*6,1:-1].values + train3.iloc[-24*7*2:-24*7*1-24*6,1:-1].values + \
         train3.iloc[-24*7*3:-24*7*2-24*6,1:-1].values + train3.iloc[-24*7*4:-24*7*3-24*6,1:-1].values)/4)\
             * 0.2 + train3.iloc[-24*1:,1:-1].values * 0.8
sub.loc[(sub['time']>='2022-07-21 01:00:00')&(sub['time']<'2022-07-22 01:00:00'),1:] = sub3
for i in range(2, 8):
    if i == 7 :
        sub.iloc[24*7*2 + 24*(i-1)*1:24*7*2 + 24*(i),1:] = ((train3.iloc[-24*1:,1:-1].values + train3.iloc[-24*7*1-24*1:-24*7*1,1:-1].values +\
            + train3.iloc[-24*7*2-24*1:-24*7*2,1:-1].values + train3.iloc[-24*7*3-24*1:-24*7*3,1:-1].values)/4)\
                 * 0.4 + sub.iloc[24*7*2 + 24*(i-2)*1:24*7*2 + 24*(i-1),1:].values * 0.6
    else:
        sub.iloc[24*7*2 + 24*(i-1)*1:24*7*2 + 24*(i),1:] = ((train3.iloc[-24*(8-i)*1:-24*(7-i),1:-1].values + train3.iloc[-24*7*1-24*(8-i)*1:-24*7*1-24*(7-i),1:-1].values +\
            train3.iloc[-24*7*2-24*(8-i)*1:-24*7*2-24*(7-i),1:-1].values + train3.iloc[-24*7*3-24*(8-i)*1:-24*7*3-24*(7-i),1:-1].values)/4)\
                 * 0.4 + sub.iloc[24*7*2 + 24*(i-2)*1:24*7*2 + 24*(i-1),1:].values * 0.6

sub4 = ((train4.iloc[-24*7*1:-24*6,1:-1].values + train4.iloc[-24*7*2:-24*7*1-24*6,1:-1].values + \
         train4.iloc[-24*7*3:-24*7*2-24*6,1:-1].values)/3)\
             * 0.2 + train4.iloc[-24*1:,1:-1].values * 0.8
sub.loc[(sub['time']>='2022-08-21 01:00:00')&(sub['time']<'2022-08-22 01:00:00'),1:] = sub4
for i in range(2, 8):
    if i == 7 :
        sub.iloc[24*7*3 + 24*(i-1)*1:24*7*3 + 24*(i),1:] = ((train4.iloc[-24*1:,1:-1].values + train4.iloc[-24*7*1-24*1:-24*7*1,1:-1].values +\
            + train4.iloc[-24*7*2-24*1:-24*7*2,1:-1].values)/3) \
                * 0.4 + sub.iloc[24*7*3 + 24*(i-2)*1:24*7*3 + 24*(i-1),1:].values * 0.6
    else:
        sub.iloc[24*7*3 + 24*(i-1)*1:24*7*3 + 24*(i),1:] = ((train4.iloc[-24*(8-i)*1:-24*(7-i),1:-1].values + train4.iloc[-24*7*1-24*(8-i)*1:-24*7*1-24*(7-i),1:-1].values +\
            train4.iloc[-24*7*2-24*(8-i)*1:-24*7*2-24*(7-i),1:-1].values)/3)\
                 * 0.4 + sub.iloc[24*7*3 + 24*(i-2)*1:24*7*3 + 24*(i-1),1:].values * 0.6
        

C:\Users\peterhuangzi\AppData\Local\Temp\ipykernel_20556\710459956.py:24: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  sub.loc[(sub['time']>='2022-05-01 01:00:00')&(sub['time']<'2022-05-02 01:00:00'),1:] = sub1
C:\Users\peterhuangzi\AppData\Local\Temp\ipykernel_20556\710459956.py:39: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  sub.loc[(sub['time']>='2022-06-01 01:00:00')&(sub['time']<'2022-06-02 01:00:00'),1:] = sub2
C:\Users\peterhuangzi\AppData\Local\Temp\ipykernel_20556\710459956.py:53: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  sub.loc[(sub['time']>='2022-07-21 01:00:00')&(sub['time']<'2022-07-22 01:00

In [7]:
import tensorflow as tf
test = pd.read_csv('use_4week_mean.csv').iloc[:,1:].values
pred = sub.iloc[:,1:].values

msle = tf.keras.losses.MeanSquaredLogarithmicError()

1/(1+msle(test, pred).numpy())

0.9783945404427217

In [8]:
sub.to_csv('test.csv',index = False)